# Github_Stars_Predictor
* Decision Tree
* Random Forests 
* Ada Boost Random Forests

In [115]:
# Import Packages
import pandas as pd
import numpy as np
import mysql.connector
import json
import graphviz 

from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.ensemble import AdaBoostClassifier


#Get Dataset through connectting to mysql
with open('../config/config_dev.json') as json_file:
    conf = json.load(json_file)

default_datatable = conf['default_table']

db = mysql.connector.connect(
    host = conf['host'],
    user = conf['user'],
    passwd = conf['password'],
    database = conf['database']
)


df = pd.read_sql(f'SELECT * FROM {default_datatable}', con=db)


In [147]:
# df.describe()
star_counts_median = df[['star_count']].median().values
df['Star_counts_label'] = [1 if x >= star_counts_median else 0 for x in df['star_count']]
df.head()

,id,full_name,forks_count,open_issues_count,watchers_count,star_count,vader_setimental_result,Good_or_Bad,Star_counts_label
0,0,Tyrrrz/DiscordChatExporter,107,24,778,778,0.3355,0,0
1,1,LorenzCK/OnTopReplica,110,61,776,776,0.9751,0,0
2,2,TheCGDF/SSD-Windows,131,5,776,776,0.0000,1,0
3,3,zkSNACKs/WalletWasabi,177,301,775,775,-0.8776,0,0
4,4,Unity-Technologies/arfoundation-samples,225,150,774,774,0.9409,1,0


In [148]:
Y = df[['Good_or_Bad']]
X = df[['forks_count','open_issues_count','watchers_count','star_count']]

In [149]:
#Split training testing datasets
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.6, random_state=80)

In [150]:
# Decision Tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.5708061002178649

In [135]:
# Store decision tree result
dot_data = tree.export_graphviz(clf, out_file=None) 
dot_data = tree.export_graphviz(clf, out_file=None, 
                      feature_names=np.array(['forks_count','open_issues_count','watchers_count','star_count']),  
                      class_names=np.array(['1', '0']),  
                      filled=True, rounded=True,  
                      special_characters=True)  
graph = graphviz.Source(dot_data)
graph = graphviz.Source(dot_data)
graph.render("./result/decision_tree")

'./result/decision_tree.pdf'

In [151]:
# Random forest
X_train_rf, Y_train_rf = make_classification(n_samples=5000, n_features=4, n_informative=2, n_redundant=0, random_state=0)
clf_rf = RandomForestClassifier(max_depth=2, random_state=0)
clf_rf.fit(X_train_rf, Y_train_rf)

y_pred_rf = clf_rf.predict(X_test)
accuracy_score(y_test, y_pred_rf)

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.5715638912569859

In [152]:
# Adaboost
X_train_ada, Y_train_ada = make_classification(n_samples=1000, n_features=4,
                           n_informative=2, n_redundant=0,
                           random_state=0, shuffle=False)

clf_ada = AdaBoostClassifier(n_estimators=100, random_state=0)
clf_ada.fit(X_train_ada, Y_train_ada)
y_pred_ada = clf_ada.predict(X_test)
accuracy_score(y_test, y_pred_ada)

0.5717533390167662